In [1]:
import collections.abc
collections.Iterable = collections.abc.Iterable
collections.Mapping = collections.abc.Mapping
collections.MutableSet = collections.abc.MutableSet
collections.MutableMapping = collections.abc.MutableMapping

import numpy as np
import scipy.stats as st
import pandas as pd
import os
import warnings
import iqplot

import bokeh.io
import bokeh.plotting
import bokeh.layouts
bokeh.io.output_notebook()

Loading BokehJS ...

### Specifying the data, models, main effects

In [2]:
filename = 'MotorBehaviordata_Auranofin_Livia_010924.xlsx'
data_path = os.path.join("..", "..", "Data", "Auranofin", filename)
sheet_name = 'Pole and Beam'

# Group factors
effect_1 = 'Genotype' 
effect_2 = 'Treatment'

# Type of test for current analysis
mtest = 'Pole_T'

# Whether or not the experiment has several cohorts and a dedicated cohort column
cohort_col = False 

### Plotting characteristics

Specifying palettes and plotting order depending on the data used (SPF/GF, NAC, Auronafin, DJKO experiments).

In [3]:
# Palettes:
palette_CI = ["#6baed6", "#08519c", "#6baed6", "#08519c"]

### Tidying up the data

In [4]:
pole_beam_df = pd.read_excel(data_path, sheet_name=sheet_name)
pole_beam_df = pole_beam_df.rename(columns={'ID ':'ID', 'Group ':'Group', 'treatment':'Treatment'})

if (effect_1 not in pole_beam_df.columns) or (effect_2 not in pole_beam_df.columns):
    pole_beam_df[[effect_1, effect_2]] = pole_beam_df['Group'].str.split('-', expand = True)

Creating the working data frame only with the values of interest, removing missing values and cases where mice jumped off of the pole/beam.

In [5]:
pole_beam_df = pole_beam_df.replace(to_replace = 'jumped', value = float("NaN"))

res = [i for i in list(pole_beam_df.columns) if (mtest) in i]

if cohort_col:
    col_list = [cohort_col, effect_1, effect_2, 'ID']
else:
    col_list = [effect_1, effect_2, 'ID']

beam_df = pole_beam_df[col_list + res].copy()
beam_df_long = beam_df.melt(id_vars = col_list, var_name='Trial', value_name='Time')
beam_df_long['Trial'] = beam_df_long['Trial'].str[-2:]
beam_df_long['ID'] = beam_df_long['ID'].astype('str')
beam_df_long = beam_df_long.dropna()

In [6]:
p = iqplot.stripbox(
    data=beam_df_long,
    q='Time',
    cats=['Treatment', 'Genotype'],
    order=[
        ("V", "WT"),
        ("V", "ASO"),
        ("A", "WT"),
        ("A", "ASO")
    ],
    q_axis='y',
    palette=palette_CI,
    spread="jitter",
    tooltips=[
        ("Genotype", "@Genotype"),
        ("Treatment", "@Treatment"),
        ("Mouse ID", "@ID"),
        ("Trial #", "@Trial")
    ]
)

p.output_backend = 'svg'
bokeh.io.export_svgs(p, filename=os.path.join("..", "Output", "auranofin_pole_all_trials.svg"))
bokeh.io.show(p)

In [8]:
%load_ext watermark
%watermark -v -p numpy,pandas,numba,bokeh,bebi103,jupyterlab

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Python implementation: CPython
Python version       : 3.13.2
IPython version      : 9.0.2

numpy     : 2.1.3
pandas    : 2.2.3
numba     : 0.61.0
bokeh     : 3.7.0
bebi103   : 0.1.26
jupyterlab: 4.3.6

